## [Bayesian Factorization Machines with Stan and R](https://nyhackr.blob.core.windows.net/presentations/A_Common_Model_Separated_by_Two_Disciplines-Adam_Lauretig.pdf)

### [Mastering Shiny](https://mastering-shiny.org/basic-intro.html)

In [116]:
library(MASS)
library(Matrix)
library(trialr)

In [117]:
library(rstan)
options(mc.cores = parallel::detectCores())
rstan_options(auto_write = TRUE)

### Generating Simulation Data

In [118]:
set.seed(111)

In [119]:
N <- 25
group_1 <- paste0('i', 1:N)

In [120]:
J <- 25
group_2 <- paste0('j', 1:J)

In [121]:
K <- 5

In [122]:
predictors <- expand.grid(group_1=group_1, group_2=group_2)

In [123]:
X_mat <- sparse.model.matrix(~ factor(group_1) + factor(group_2) -1, data=predictors)

In [124]:
predictors_as_numeric <- cbind(as.numeric(factor(predictors[,1])), as.numeric(factor(predictors[,2])))

In [125]:
betas <- matrix(rnorm(n=ncol(X_mat), 0,1))

In [126]:
linear_predictors = X_mat %*% betas  # y

### [LKJ](https://yingqijing.medium.com/lkj-correlation-distribution-in-stan-29927b69e9be)

#### if eta > 1, the correlation values in correlation matrices are going to centered around 0. higher eta indicate no correlations (converge to identity correlation matrix).

![](https://miro.medium.com/max/1400/1*v2LSgSMjYwCjWYpXtk437g.webp)

In [127]:
gamma_omega <- rlkjcorr(n = 1, K = K, eta = 0.5)#gamma_omega_prior)

In [128]:
delta_omega <- rlkjcorr(n=1, K=K, eta=0.5)

In [129]:
gammas <- mvrnorm(n=N, mu=rep(0,K), Sigma = gamma_omega)

In [130]:
deltas <- mvrnorm(n=J, mu=rep(0,K), Sigma = delta_omega)

In [131]:
factor_terms = matrix(NA, nrow=nrow(linear_predictors), ncol=1)

In [132]:
dim(factor_terms)

[1] 625   1

In [133]:
for (i in 1:nrow(predictors)){
    g1 <- as.character(predictors[i,1])
    g1 <- as.numeric(substring(g1, 2, nchar(g1)))

    g2 <- as.character(predictors[i,2])
    g2 <- as.numeric(substring(g2, 2, nchar(g2)))

    #print(paste0(g1, g2))
    factor_terms[i, ] =  matrix(gammas[g1,], nrow=1) %*% matrix(deltas[g2,], ncol=1)   
}

In [134]:
y <- linear_predictors + factor_terms + rnorm(n=nrow(linear_predictors), 0, 0.1)

In [135]:
data_list = list(
    N = N,
    J = J,
    K = K,
    X = predictors_as_numeric,
    y = y[,1],
    beta_sigma = 1,
    y_sigma = 0.1
)

In [136]:
saveRDS(data_list, file = 'data_list.rds')

In [137]:
model = stan_model('../RStan/Models/factorization.stan')

In [138]:
remove(fit)
fit <- sampling(object = model,
                data = data_list,
                init = "random",
                control = list(adapt_delta = 0.95),
                chains = 4,
                iter = 10,
                warmup = 5,
                thin = 1,
                verbose = TRUE)


CHECKING DATA AND PREPROCESSING FOR MODEL 'factorization' NOW.

COMPILING MODEL 'factorization' NOW.

STARTING SAMPLER FOR MODEL 'factorization' NOW.


Warning message:
"There were 20 divergent transitions after warmup. See
https://mc-stan.org/misc/warnings.html#divergent-transitions-after-warmup
to find out why this is a problem and how to eliminate them."
Warning message:
"There were 1 chains where the estimated Bayesian Fraction of Missing Information was low. See
https://mc-stan.org/misc/warnings.html#bfmi-low"
Warning message:
"Examine the pairs() plot to diagnose sampling problems
"
Warning message:
"The largest R-hat is Inf, indicating chains have not mixed.
Running the chains for more iterations may help. See
https://mc-stan.org/misc/warnings.html#r-hat"
